In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from matplotlib import pyplot as plt
from matplotlib import rcParams
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('Time_series_data.csv')
pd.reset_option('display.max_rows', None)
df

,Date,Ser.No.,Age,Sex,Address,1st complaint,2nd Complaint,3rd Complaint
0,2011-05-01,1.0,18,2,KOT ABDUL MALIK,fever,bodyaches,Headache
1,2011-05-01,2.0,40,1,GUJRAT,bleeding,pain,RTA
2,2011-05-01,3.0,20,2,GULSHAN RAVI,pain,bleeding,RTA
3,2011-05-01,4.0,18,2,SAMANABAD,Pain,Anxiety,Dogbite
4,2011-05-01,5.0,40,2,GOWALMANDI,Dyspnea,chest pain,RTI
...,...,...,...,...,...,...,...,...
133393,2011-08-31,40610.0,35,1,ANARKALI,Vomiting,fever,nausea
133394,2011-08-31,40611.0,35,1,"GOWALMANDI,LAHORE",Arthritis,Anxiety,arthralgia
133395,2011-08-31,40612.0,18,2,FAROOZWALA,Abdomen pain,Nausea,nausea
133396,2011-08-31,40613.0,46,1,LOHARIGAT,Palpitation,anxiety,arthralgia


In [3]:
df_address = df[['Address']]
df_remain = df[['Date', 'Age', 'Sex', '1st complaint', '2nd Complaint', '3rd Complaint']]
df_address_unique = df_address.value_counts()

In [4]:
df_address_unique

Address           
GOWALMANDI,LAHORE     35247
NISBAT ROAD,LAHORE     4562
SHEKHUPURA             3094
SHAHDARA LHR           1803
SHAHDARA               1525
                      ...  
BEGAM KO LHR              1
BEEKU ROAD                1
BEEKO                     1
BEDOR RD LHR              1
SHADRA FAROZ WALA         1
Length: 13492, dtype: int64

In [5]:
threshold = 50
to_remove = df_address_unique[df_address_unique <= threshold].index
df_address.replace(to_remove, np.nan, inplace=True)

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\2509391112.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_address.replace(to_remove, np.nan, inplace=True)


In [6]:
df_address

,Address
0,KOT ABDUL MALIK
1,NaN
2,GULSHAN RAVI
3,NaN
4,GOWALMANDI
...,...
133393,ANARKALI
133394,"GOWALMANDI,LAHORE"
133395,FAROOZWALA
133396,LOHARIGAT


In [7]:
df = pd.concat([df_remain, df_address], axis=1)
df

,Date,Age,Sex,1st complaint,2nd Complaint,3rd Complaint,Address
0,2011-05-01,18,2,fever,bodyaches,Headache,KOT ABDUL MALIK
1,2011-05-01,40,1,bleeding,pain,RTA,NaN
2,2011-05-01,20,2,pain,bleeding,RTA,GULSHAN RAVI
3,2011-05-01,18,2,Pain,Anxiety,Dogbite,NaN
4,2011-05-01,40,2,Dyspnea,chest pain,RTI,GOWALMANDI
...,...,...,...,...,...,...,...
133393,2011-08-31,35,1,Vomiting,fever,nausea,ANARKALI
133394,2011-08-31,35,1,Arthritis,Anxiety,arthralgia,"GOWALMANDI,LAHORE"
133395,2011-08-31,18,2,Abdomen pain,Nausea,nausea,FAROOZWALA
133396,2011-08-31,46,1,Palpitation,anxiety,arthralgia,LOHARIGAT


In [8]:
df['Address'].value_counts()

GOWALMANDI,LAHORE     35247
NISBAT ROAD,LAHORE     4562
SHEKHUPURA             3094
SHAHDARA LHR           1803
SHAHDARA               1525
                      ...  
JIA MUSA SHADRA          52
IQBAL TOWN               52
DAROOGANWALA             51
DROUGHA WALA LHR         51
BHATI GATE LHR           51
Name: Address, Length: 248, dtype: int64

In [9]:
df.isna().sum() 

Date                 0
Age                  0
Sex                  0
1st complaint        0
2nd Complaint        0
3rd Complaint        0
Address          49940
dtype: int64

In [10]:
df = df.dropna(subset=['Address'])
df.reset_index(drop = True, inplace = True)
df

,Date,Age,Sex,1st complaint,2nd Complaint,3rd Complaint,Address
0,2011-05-01,18,2,fever,bodyaches,Headache,KOT ABDUL MALIK
1,2011-05-01,20,2,pain,bleeding,RTA,GULSHAN RAVI
2,2011-05-01,40,2,Dyspnea,chest pain,RTI,GOWALMANDI
3,2011-05-01,24,2,headache,allergy,Vomiting,KOT ABDUL MALIK
4,2011-05-01,26,1,bleeding,pain,RTA,URDU BAZAR
...,...,...,...,...,...,...,...
83453,2011-08-31,35,1,Vomiting,fever,nausea,ANARKALI
83454,2011-08-31,35,1,Arthritis,Anxiety,arthralgia,"GOWALMANDI,LAHORE"
83455,2011-08-31,18,2,Abdomen pain,Nausea,nausea,FAROOZWALA
83456,2011-08-31,46,1,Palpitation,anxiety,arthralgia,LOHARIGAT


In [11]:
df_address = df['Address']

In [12]:
pd.set_option('display.max_rows', None)
df_address = df_address.value_counts()
df_address

GOWALMANDI,LAHORE                35247
NISBAT ROAD,LAHORE                4562
SHEKHUPURA                        3094
SHAHDARA LHR                      1803
SHAHDARA                          1525
BADAMI BAGH LHR                    973
MOZANG,LAHORE                      924
MISRISHAH,LAHORE                   870
MUGHALPURA,LAHORE                  762
GUWAL MANDI LHR                    751
RAVI RD LHR                        705
SHAHDRA                            629
LOHARI GATE LHR                    590
GAWAL MANDI LHR                    485
LOHARI GATE                        465
BADAMI BAGH                        450
SHAD BAGH LHR                      447
BHATTI GATE                        400
BILAL GUNJ LHR                     389
SANDA BAND RD LHR                  386
GUJRAWALA                          379
ANARKALI LHR                       367
RAVI RD                            345
SHAHDARA TOWN LHR                  343
BAND RD LHR                        343
KOT ABDUL MALIK          

In [13]:
df['Latitude'] = np.nan
df['Longitude'] = np.nan

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\3469199155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Latitude'] = np.nan
C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\3469199155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Longitude'] = np.nan


In [14]:
pd.reset_option('display.max_rows', None)
df

,Date,Age,Sex,1st complaint,2nd Complaint,3rd Complaint,Address,Latitude,Longitude
0,2011-05-01,18,2,fever,bodyaches,Headache,KOT ABDUL MALIK,NaN,NaN
1,2011-05-01,20,2,pain,bleeding,RTA,GULSHAN RAVI,NaN,NaN
2,2011-05-01,40,2,Dyspnea,chest pain,RTI,GOWALMANDI,NaN,NaN
3,2011-05-01,24,2,headache,allergy,Vomiting,KOT ABDUL MALIK,NaN,NaN
4,2011-05-01,26,1,bleeding,pain,RTA,URDU BAZAR,NaN,NaN
...,...,...,...,...,...,...,...,...,...
83453,2011-08-31,35,1,Vomiting,fever,nausea,ANARKALI,NaN,NaN
83454,2011-08-31,35,1,Arthritis,Anxiety,arthralgia,"GOWALMANDI,LAHORE",NaN,NaN
83455,2011-08-31,18,2,Abdomen pain,Nausea,nausea,FAROOZWALA,NaN,NaN
83456,2011-08-31,46,1,Palpitation,anxiety,arthralgia,LOHARIGAT,NaN,NaN


In [15]:
df['Address'] = df['Address'].replace(['GOWALMANDI,LAHORE', 'GUWAL MANDI LHR', 'GAWAL MANDI LHR', 'GUWALMANDI LHR', 'GOWALMANDI', 'GAWALMANDI LHR', 'GUWALMANDI', 'GUWAL MANDI', 'GOWALMANDI ', 'GAWAL MANDI LHR '], 'Gawalmandi')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\3444027939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['GOWALMANDI,LAHORE', 'GUWAL MANDI LHR', 'GAWAL MANDI LHR', 'GUWALMANDI LHR', 'GOWALMANDI', 'GAWALMANDI LHR', 'GUWALMANDI', 'GUWAL MANDI', 'GOWALMANDI ', 'GAWAL MANDI LHR '], 'Gawalmandi')


In [16]:
mask = ((df['Address'] == 'Gawalmandi'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.57187)

In [17]:
mask = ((df['Address']== 'Gawalmandi'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.31826)

In [18]:
df

,Date,Age,Sex,1st complaint,2nd Complaint,3rd Complaint,Address,Latitude,Longitude
0,2011-05-01,18,2,fever,bodyaches,Headache,KOT ABDUL MALIK,NaN,NaN
1,2011-05-01,20,2,pain,bleeding,RTA,GULSHAN RAVI,NaN,NaN
2,2011-05-01,40,2,Dyspnea,chest pain,RTI,Gawalmandi,31.57187,74.31826
3,2011-05-01,24,2,headache,allergy,Vomiting,KOT ABDUL MALIK,NaN,NaN
4,2011-05-01,26,1,bleeding,pain,RTA,URDU BAZAR,NaN,NaN
...,...,...,...,...,...,...,...,...,...
83453,2011-08-31,35,1,Vomiting,fever,nausea,ANARKALI,NaN,NaN
83454,2011-08-31,35,1,Arthritis,Anxiety,arthralgia,Gawalmandi,31.57187,74.31826
83455,2011-08-31,18,2,Abdomen pain,Nausea,nausea,FAROOZWALA,NaN,NaN
83456,2011-08-31,46,1,Palpitation,anxiety,arthralgia,LOHARIGAT,NaN,NaN


In [19]:
pd.set_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi                       38095
NISBAT ROAD,LAHORE                4562
SHEKHUPURA                        3094
SHAHDARA LHR                      1803
SHAHDARA                          1525
BADAMI BAGH LHR                    973
MOZANG,LAHORE                      924
MISRISHAH,LAHORE                   870
MUGHALPURA,LAHORE                  762
RAVI RD LHR                        705
SHAHDRA                            629
LOHARI GATE LHR                    590
LOHARI GATE                        465
BADAMI BAGH                        450
SHAD BAGH LHR                      447
BHATTI GATE                        400
BILAL GUNJ LHR                     389
SANDA BAND RD LHR                  386
GUJRAWALA                          379
ANARKALI LHR                       367
RAVI RD                            345
SHAHDARA TOWN LHR                  343
BAND RD LHR                        343
KOT ABDUL MALIK                    337
BHATI GATE LHR                     333
NAROWAL                  

In [20]:
df['Address'] = df['Address'].replace(['GUJRAWALA', 'GUJRANWALA', 'GUJRANWALA ', 'GUJJRANWALA', 'GUJRAWALA ', 'ZILA GUJRANWALA '], 'Gujranwala')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\3982368643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['GUJRAWALA', 'GUJRANWALA', 'GUJRANWALA ', 'GUJJRANWALA', 'GUJRAWALA ', 'ZILA GUJRANWALA '], 'Gujranwala')


In [21]:
mask = ((df['Address'] == 'Gujranwala'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(32.166351)

In [22]:
mask = ((df['Address'] == 'Gujranwala'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.195900)

In [23]:
df['Address'] = df['Address'].replace(['SHAHDARA LHR', 'SHAHDARA ', 'SHAHDRA ', 'SHAHDARA TOWN LHR', 'SHADRA MOR LHR ', 'SHAHDRA LHR ', 'SHAHDRA TOWN LHR ', 'SHADRA TOWN LHR', 'SHAHDARA STATION LHR', 'SHAHDARA', 'SHAHDRA', 'SHAHDRA LHR', 'SHAHDRA TOWN LHR', 'SHADRA TOWN LHR '], 'Shahdara')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\609958816.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['SHAHDARA LHR', 'SHAHDARA ', 'SHAHDRA ', 'SHAHDARA TOWN LHR', 'SHADRA MOR LHR ', 'SHAHDRA LHR ', 'SHAHDRA TOWN LHR ', 'SHADRA TOWN LHR', 'SHAHDARA STATION LHR', 'SHAHDARA', 'SHAHDRA', 'SHAHDRA LHR', 'SHAHDRA TOWN LHR', 'SHADRA TOWN LHR '], 'Shahdara')


In [24]:
mask = ((df['Address'] == 'Shahdara'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.63212)

In [25]:
mask = ((df['Address'] == 'Shahdara'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.29863)

In [26]:
pd.set_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi                       38095
Shahdara                          5429
NISBAT ROAD,LAHORE                4562
SHEKHUPURA                        3094
Gujranwala                        1405
BADAMI BAGH LHR                    973
MOZANG,LAHORE                      924
MISRISHAH,LAHORE                   870
MUGHALPURA,LAHORE                  762
RAVI RD LHR                        705
LOHARI GATE LHR                    590
LOHARI GATE                        465
BADAMI BAGH                        450
SHAD BAGH LHR                      447
BHATTI GATE                        400
BILAL GUNJ LHR                     389
SANDA BAND RD LHR                  386
ANARKALI LHR                       367
RAVI RD                            345
BAND RD LHR                        343
KOT ABDUL MALIK                    337
BHATI GATE LHR                     333
NAROWAL                            323
GULSHAN RAVI LHR                   320
SANT NAGAR LHR                     315
ICCHRA,LAHORE            

In [27]:
df['Address'] = df['Address'].replace(['NISBAT ROAD,LAHORE', 'NISBAT RD LHR'], 'Nisbat Road')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\481751833.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['NISBAT ROAD,LAHORE', 'NISBAT RD LHR'], 'Nisbat Road')


In [28]:
mask = ((df['Address'] == 'Nisbat Road'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5698973)

In [29]:
mask = ((df['Address'] == 'Nisbat Road'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.32021)

In [30]:
df['Address'].value_counts()

Gawalmandi                       38095
Shahdara                          5429
Nisbat Road                       4686
SHEKHUPURA                        3094
Gujranwala                        1405
BADAMI BAGH LHR                    973
MOZANG,LAHORE                      924
MISRISHAH,LAHORE                   870
MUGHALPURA,LAHORE                  762
RAVI RD LHR                        705
LOHARI GATE LHR                    590
LOHARI GATE                        465
BADAMI BAGH                        450
SHAD BAGH LHR                      447
BHATTI GATE                        400
BILAL GUNJ LHR                     389
SANDA BAND RD LHR                  386
ANARKALI LHR                       367
RAVI RD                            345
BAND RD LHR                        343
KOT ABDUL MALIK                    337
BHATI GATE LHR                     333
NAROWAL                            323
GULSHAN RAVI LHR                   320
SANT NAGAR LHR                     315
ICCHRA,LAHORE            

In [31]:
df['Address'] = df['Address'].replace(['SHEKHUPURA'], 'sheikhupura')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\1276274972.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['SHEKHUPURA'], 'sheikhupura')


In [32]:
mask = ((df['Address'] == 'sheikhupura'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.716661)

In [33]:
mask = ((df['Address'] == 'sheikhupura'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(73.985023)

In [34]:
pd.reset_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi                  38095
Shahdara                     5429
Nisbat Road                  4686
sheikhupura                  3094
Gujranwala                   1405
                            ...  
JIA MUSA SHADRA                52
MUREED KAY SHEIKHU PURA        52
BHATI GATE LHR                 51
DROUGHA WALA LHR               51
DAROOGANWALA                   51
Name: Address, Length: 224, dtype: int64

In [35]:
df['Address'] = df['Address'].replace(['BADAMI BAGH LHR', 'BADAMI BAGH', 'BADAMIBAG', 'BADMI BAGH LHR', 'BADAMI BAGH ', 'BADMAI BAGH LHR ', 'BADAMI BAGH LHR '], 'Badami Bagh')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\641022979.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['BADAMI BAGH LHR', 'BADAMI BAGH', 'BADAMIBAG', 'BADMI BAGH LHR', 'BADAMI BAGH ', 'BADMAI BAGH LHR ', 'BADAMI BAGH LHR '], 'Badami Bagh')


In [36]:
mask = ((df['Address'] == 'Badami Bagh'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.601849)

In [37]:
mask = ((df['Address'] == 'Badami Bagh'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.320621)

In [38]:
pd.reset_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi                  38095
Shahdara                     5429
Nisbat Road                  4686
sheikhupura                  3094
Badami Bagh                  2301
                            ...  
JIA MUSA SHADRA                52
MUREED KAY SHEIKHU PURA        52
DROUGHA WALA LHR               51
DAROOGANWALA                   51
BHATI GATE LHR                 51
Name: Address, Length: 218, dtype: int64

In [39]:
mask = ((df['Address'] == 'MOZANG,LAHORE'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5597)

In [40]:
mask = ((df['Address'] == 'MOZANG,LAHORE'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3138)

In [41]:
mask = ((df['Address'] == 'MISRISHAH,LAHORE'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.593401)

In [42]:
mask = ((df['Address'] == 'MISRISHAH,LAHORE'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.330886)

In [43]:
mask = ((df['Address'] == 'MUGHALPURA,LAHORE'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5690)

In [44]:
mask = ((df['Address'] == 'MUGHALPURA,LAHORE'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3586)

In [45]:
df['Address'] = df['Address'].replace(['RAVI RD LHR', 'RAVI RD', 'RAVI ROAD ', 'RAVI ROAD LHR', 'RAVI RD LHR ', 'RAVI RD '], 'Ravi Road, Lahore')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\2144283667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['RAVI RD LHR', 'RAVI RD', 'RAVI ROAD ', 'RAVI ROAD LHR', 'RAVI RD LHR ', 'RAVI RD '], 'Ravi Road, Lahore')


In [46]:
mask = ((df['Address'] == 'Ravi Road, Lahore'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5497)

mask = ((df['Address'] == 'Ravi Road, Lahore'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3436)

In [47]:
pd.reset_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi           38095
Shahdara              5429
Nisbat Road           4686
sheikhupura           3094
Badami Bagh           2301
                     ...  
SHAH ALAMI GATE         52
JIA MUSA SHADRA         52
DAROOGANWALA            51
DROUGHA WALA LHR        51
BHATI GATE LHR          51
Name: Address, Length: 213, dtype: int64

In [48]:
df['Address'] = df['Address'].replace(['LOHARI GATE LHR', 'LOHARI GATE', 'LOHARI GATE LHR ', 'LOHARIGAT', 'LOHARI GATE ', 'ANDROON LOHARI GATE LHR', 'INSIDE LOHARI GATE '], 'Lahori Gate')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\1421389649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['LOHARI GATE LHR', 'LOHARI GATE', 'LOHARI GATE LHR ', 'LOHARIGAT', 'LOHARI GATE ', 'ANDROON LOHARI GATE LHR', 'INSIDE LOHARI GATE '], 'Lahori Gate')


In [49]:
mask = ((df['Address'] == 'Lahori Gate'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.57741)

mask = ((df['Address'] == 'Lahori Gate'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.31343)

In [50]:
pd.reset_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi           38095
Shahdara              5429
Nisbat Road           4686
sheikhupura           3094
Badami Bagh           2301
                     ...  
SHAH ALAMI GATE         52
JIA MUSA SHADRA         52
DAROOGANWALA            51
DROUGHA WALA LHR        51
BHATI GATE LHR          51
Name: Address, Length: 207, dtype: int64

In [51]:
df['Address'] = df['Address'].replace(['SHAD BAGH LHR', 'SHAD BAGH', 'SHAD BAGH LHR ', 'SHAD BAGH ', 'SHAHDBAG'], 'Shadbagh')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\693854414.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['SHAD BAGH LHR', 'SHAD BAGH', 'SHAD BAGH LHR ', 'SHAD BAGH ', 'SHAHDBAG'], 'Shadbagh')


In [52]:
mask = ((df['Address'] == 'Shadbagh'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.60010)

mask = ((df['Address'] == 'Shadbagh'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3397)

In [53]:
pd.reset_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi           38095
Shahdara              5429
Nisbat Road           4686
sheikhupura           3094
Badami Bagh           2301
                     ...  
SHAH ALAMI GATE         52
JIA MUSA SHADRA         52
DAROOGANWALA            51
DROUGHA WALA LHR        51
BHATI GATE LHR          51
Name: Address, Length: 203, dtype: int64

In [54]:
df['Address'] = df['Address'].replace(['BHATTI GATE ', 'BHATI GATE LHR', 'BATI GATE LHR ', 'BHATIGAT', 'BAHTI GATE LHR', 'BHATTI GATE', 'BATI GATE LHR'], 'Bhati Gate')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\1161518049.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['BHATTI GATE ', 'BHATI GATE LHR', 'BATI GATE LHR ', 'BHATIGAT', 'BAHTI GATE LHR', 'BHATTI GATE', 'BATI GATE LHR'], 'Bhati Gate')


In [55]:
mask = ((df['Address'] == 'Bhati Gate'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5802)

mask = ((df['Address'] == 'Bhati Gate'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3073)

In [56]:
pd.reset_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi           38095
Shahdara              5429
Nisbat Road           4686
sheikhupura           3094
Badami Bagh           2301
                     ...  
JIA MUSA SHADRA         52
IQBAL TOWN              52
DAROOGANWALA            51
DROUGHA WALA LHR        51
BHATI GATE LHR          51
Name: Address, Length: 198, dtype: int64

In [57]:
df['Address'] = df['Address'].replace(['BILAL GUNJ LHR', 'BILAL GANJ ', 'BILAL GANJ'], 'Bilal Ganj Lahore')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\2671291057.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['BILAL GUNJ LHR', 'BILAL GANJ ', 'BILAL GANJ'], 'Bilal Ganj Lahore')


In [58]:
mask = ((df['Address'] == 'Bilal Ganj Lahore'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5820)

mask = ((df['Address'] == 'Bilal Ganj Lahore'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3008)

In [59]:
pd.reset_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi           38095
Shahdara              5429
Nisbat Road           4686
sheikhupura           3094
Badami Bagh           2301
                     ...  
SHAH ALAMI GATE         52
JIA MUSA SHADRA         52
DAROOGANWALA            51
DROUGHA WALA LHR        51
BHATI GATE LHR          51
Name: Address, Length: 197, dtype: int64

In [60]:
df['Address'] = df['Address'].replace(['SANDA BAND RD LHR', 'SANDA BAND RD LHR ', 'SANDAH ', 'SANDA', 'SANDA ', 'SANDA KALAN LHR', 'SANDA KALAN LHR ', 'SANDA LHR'], 'Sanda Lahore')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\913642310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['SANDA BAND RD LHR', 'SANDA BAND RD LHR ', 'SANDAH ', 'SANDA', 'SANDA ', 'SANDA KALAN LHR', 'SANDA KALAN LHR ', 'SANDA LHR'], 'Sanda Lahore')


In [61]:
mask = ((df['Address'] == 'Sanda Lahore'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.561085)

mask = ((df['Address'] == 'Sanda Lahore'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.283496)

In [62]:
pd.set_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi                       38095
Shahdara                          5429
Nisbat Road                       4686
sheikhupura                       3094
Badami Bagh                       2301
Lahori Gate                       1892
Ravi Road, Lahore                 1811
Sanda Lahore                      1572
Gujranwala                        1405
Bhati Gate                        1216
Shadbagh                          1082
MOZANG,LAHORE                      924
MISRISHAH,LAHORE                   870
MUGHALPURA,LAHORE                  762
Bilal Ganj Lahore                  534
ANARKALI LHR                       367
BAND RD LHR                        343
KOT ABDUL MALIK                    337
NAROWAL                            323
GULSHAN RAVI LHR                   320
SANT NAGAR LHR                     315
ICCHRA,LAHORE                      309
NAROWAL                            277
RANG MEHAL LHR                     268
BAND ROAD                          266
KOT ABDUL MALIK          

In [63]:
df['Address'] = df['Address'].replace(['ANARKALI LHR', 'ANARKALI', 'NEW ANARKALI LHR', 'ANAR KALI LHR '], 'Anarkali Bazaar Lahore')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\3042211527.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['ANARKALI LHR', 'ANARKALI', 'NEW ANARKALI LHR', 'ANAR KALI LHR '], 'Anarkali Bazaar Lahore')


In [64]:
mask = ((df['Address'] == 'Anarkali Bazaar Lahore'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5698)

mask = ((df['Address'] == 'Anarkali Bazaar Lahore'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3120)

In [65]:
pd.set_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi                       38095
Shahdara                          5429
Nisbat Road                       4686
sheikhupura                       3094
Badami Bagh                       2301
Lahori Gate                       1892
Ravi Road, Lahore                 1811
Sanda Lahore                      1572
Gujranwala                        1405
Bhati Gate                        1216
Shadbagh                          1082
MOZANG,LAHORE                      924
MISRISHAH,LAHORE                   870
Anarkali Bazaar Lahore             824
MUGHALPURA,LAHORE                  762
Bilal Ganj Lahore                  534
BAND RD LHR                        343
KOT ABDUL MALIK                    337
NAROWAL                            323
GULSHAN RAVI LHR                   320
SANT NAGAR LHR                     315
ICCHRA,LAHORE                      309
NAROWAL                            277
RANG MEHAL LHR                     268
BAND ROAD                          266
KOT ABDUL MALIK          

In [66]:
df['Address'] = df['Address'].replace(['BAND RD LHR', 'BAND ROAD ', 'BAND RD '], 'Band Rd Lahore')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\2825144623.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['BAND RD LHR', 'BAND ROAD ', 'BAND RD '], 'Band Rd Lahore')


In [67]:
mask = ((df['Address'] == 'Band Rd Lahore'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.53696)

mask = ((df['Address'] == 'Band Rd Lahore'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.271942)

In [68]:
pd.set_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi                       38095
Shahdara                          5429
Nisbat Road                       4686
sheikhupura                       3094
Badami Bagh                       2301
Lahori Gate                       1892
Ravi Road, Lahore                 1811
Sanda Lahore                      1572
Gujranwala                        1405
Bhati Gate                        1216
Shadbagh                          1082
MOZANG,LAHORE                      924
MISRISHAH,LAHORE                   870
Anarkali Bazaar Lahore             824
MUGHALPURA,LAHORE                  762
Band Rd Lahore                     719
Bilal Ganj Lahore                  534
KOT ABDUL MALIK                    337
NAROWAL                            323
GULSHAN RAVI LHR                   320
SANT NAGAR LHR                     315
ICCHRA,LAHORE                      309
NAROWAL                            277
RANG MEHAL LHR                     268
KOT ABDUL MALIK                    227
KARIM PARK RAVI RD LHR   

In [69]:
df['Address'] = df['Address'].replace(['KOT ABDUL MALIK', 'KOT ABDUL MALIK '], 'KOT ABDUL MALIK')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\266560494.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['KOT ABDUL MALIK', 'KOT ABDUL MALIK '], 'KOT ABDUL MALIK')


In [70]:
mask = ((df['Address'] == 'KOT ABDUL MALIK'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.62042)

mask = ((df['Address'] == 'KOT ABDUL MALIK'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.234381)

In [71]:
df['Address'] = df['Address'].replace(['NAROWAL', 'NAROWAL '], 'Narowal')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\518359286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['NAROWAL', 'NAROWAL '], 'Narowal')


In [72]:
mask = ((df['Address'] == 'Narowal'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(32.101407)

mask = ((df['Address'] == 'Narowal'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.879952)

In [73]:
pd.set_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi                       38095
Shahdara                          5429
Nisbat Road                       4686
sheikhupura                       3094
Badami Bagh                       2301
Lahori Gate                       1892
Ravi Road, Lahore                 1811
Sanda Lahore                      1572
Gujranwala                        1405
Bhati Gate                        1216
Shadbagh                          1082
MOZANG,LAHORE                      924
MISRISHAH,LAHORE                   870
Anarkali Bazaar Lahore             824
MUGHALPURA,LAHORE                  762
Band Rd Lahore                     719
Narowal                            600
KOT ABDUL MALIK                    564
Bilal Ganj Lahore                  534
GULSHAN RAVI LHR                   320
SANT NAGAR LHR                     315
ICCHRA,LAHORE                      309
RANG MEHAL LHR                     268
KARIM PARK RAVI RD LHR             201
URDU BAZAR LHR                     199
SIALKOT                  

In [74]:
df['Address'] = df['Address'].replace(['GULSHAN RAVI LHR', 'GULSHAN RAVI', 'GULSHAN RAVI '], 'Gulshan-e-Ravi')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\2217518402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['GULSHAN RAVI LHR', 'GULSHAN RAVI', 'GULSHAN RAVI '], 'Gulshan-e-Ravi')


In [75]:
mask = ((df['Address'] == 'Gulshan-e-Ravi'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.55217)

mask = ((df['Address'] == 'Gulshan-e-Ravi'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.27529)

In [76]:
pd.reset_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi           38095
Shahdara              5429
Nisbat Road           4686
sheikhupura           3094
Badami Bagh           2301
                     ...  
SHAH ALAMI GATE         52
IQBAL TOWN              52
BHATI GATE LHR          51
DROUGHA WALA LHR        51
DAROOGANWALA            51
Name: Address, Length: 182, dtype: int64

In [77]:
df['Address'] = df['Address'].replace(['SANT NAGAR LHR', 'SUNT NAGAR LHR'], 'Sant Nagar St Nagar')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\1533144980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['SANT NAGAR LHR', 'SUNT NAGAR LHR'], 'Sant Nagar St Nagar')


In [78]:
mask = ((df['Address'] == 'Sant Nagar St Nagar'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.569521)

mask = ((df['Address'] == 'Sant Nagar St Nagar'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.295688)

In [79]:
pd.reset_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi           38095
Shahdara              5429
Nisbat Road           4686
sheikhupura           3094
Badami Bagh           2301
                     ...  
SHAH ALAMI GATE         52
IQBAL TOWN              52
BHATI GATE LHR          51
DROUGHA WALA LHR        51
DAROOGANWALA            51
Name: Address, Length: 181, dtype: int64

In [80]:
mask = ((df['Address'] == 'ICCHRA,LAHORE'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5313)

mask = ((df['Address'] == 'ICCHRA,LAHORE'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3183)

In [81]:
df['Address'] = df['Address'].replace(['RANG MEHAL LHR', 'RANG MAHAL', 'RANG MAHAL '], 'RANG MAHAL')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\3102802698.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['RANG MEHAL LHR', 'RANG MAHAL', 'RANG MAHAL '], 'RANG MAHAL')


In [82]:
mask = ((df['Address'] == 'RANG MAHAL'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5899)

mask = ((df['Address'] == 'RANG MAHAL'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3250)

In [83]:
pd.reset_option('display.max_rows', None)
df['Address'].value_counts()

Gawalmandi                  38095
Shahdara                     5429
Nisbat Road                  4686
sheikhupura                  3094
Badami Bagh                  2301
                            ...  
SAMAN ABAD LHR                 52
MUREED KAY SHEIKHU PURA        52
BHATI GATE LHR                 51
DROUGHA WALA LHR               51
DAROOGANWALA                   51
Name: Address, Length: 179, dtype: int64

In [84]:
mask = ((df['Address'] == 'KARIM PARK RAVI RD LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.593003)

mask = ((df['Address'] == 'KARIM PARK RAVI RD LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.30309)

In [85]:
mask = ((df['Address'] == 'URDU BAZAR LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5761)

mask = ((df['Address'] == 'URDU BAZAR LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3091)

In [86]:
df['Address'] = df['Address'].replace(['SIALKOT', 'SIALKOT ', 'ZILA SIALKOT '], 'Sialkot')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\2530059787.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['SIALKOT', 'SIALKOT ', 'ZILA SIALKOT '], 'Sialkot')


In [87]:
mask = ((df['Address'] == 'Sialkot'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(32.494499)

mask = ((df['Address'] == 'Sialkot'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.522892)

In [88]:
df['Address'].value_counts()

Gawalmandi                  38095
Shahdara                     5429
Nisbat Road                  4686
sheikhupura                  3094
Badami Bagh                  2301
                            ...  
SAMAN ABAD LHR                 52
MUREED KAY SHEIKHU PURA        52
BHATI GATE LHR                 51
DROUGHA WALA LHR               51
DAROOGANWALA                   51
Name: Address, Length: 177, dtype: int64

In [89]:
mask = ((df['Address'] == 'OKARA '))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(30.813802)

mask = ((df['Address'] == 'OKARA '))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(73.453378)

In [90]:
df['Address'] = df['Address'].replace(['BAGH BAN PURA LHR', 'BAGH BAN PURA '], 'Baghbanpura')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\2648692039.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['BAGH BAN PURA LHR', 'BAGH BAN PURA '], 'Baghbanpura')


In [91]:
mask = ((df['Address'] == 'Baghbanpura'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.584991)

mask = ((df['Address'] == 'Baghbanpura'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.374875)

In [92]:
df['Address'].value_counts()

Gawalmandi                  38095
Shahdara                     5429
Nisbat Road                  4686
sheikhupura                  3094
Badami Bagh                  2301
                            ...  
JIA MUSA SHADRA                52
MUREED KAY SHEIKHU PURA        52
BHATI GATE LHR                 51
DROUGHA WALA LHR               51
DAROOGANWALA                   51
Name: Address, Length: 176, dtype: int64

In [93]:
df.isna().sum()

Date                 0
Age                  0
Sex                  0
1st complaint        0
2nd Complaint        0
3rd Complaint        0
Address              0
Latitude         11817
Longitude        11817
dtype: int64

In [94]:
mask = ((df['Address'] == 'RAILWAY STATION LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5773)

mask = ((df['Address'] == 'RAILWAY STATION LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3363)

In [95]:
mask = ((df['Address'] == 'PANI WAL ATALAB ,LAHORE'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.584427)

mask = ((df['Address'] == 'PANI WAL ATALAB ,LAHORE'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3163)

In [96]:
mask = ((df['Address'] == 'KARIM PARK LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5894)

mask = ((df['Address'] == 'KARIM PARK LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.2986)

In [97]:
df['Address'] = df['Address'].replace(['SHALMI GATE LHR ', 'SHAH ALAMI GATE ', 'SHAH ALAM GATE ', 'SHALMI GATE LHR', 'SHAH ALAMI GATE', 'SHAH ALAM GATE'], 'Shah Alami')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\4093800428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['SHALMI GATE LHR ', 'SHAH ALAMI GATE ', 'SHAH ALAM GATE ', 'SHALMI GATE LHR', 'SHAH ALAMI GATE', 'SHAH ALAM GATE'], 'Shah Alami')


In [98]:
mask = ((df['Address'] == 'Shah Alami'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5777)

mask = ((df['Address'] == 'Shah Alami'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3205)

In [99]:
df['Address'].value_counts()

Gawalmandi                  38095
Shahdara                     5429
Nisbat Road                  4686
sheikhupura                  3094
Badami Bagh                  2301
                            ...  
JIA MUSA SHADRA                52
MUREED KAY SHEIKHU PURA        52
BHATI GATE LHR                 51
DAROOGANWALA                   51
DROUGHA WALA LHR               51
Name: Address, Length: 173, dtype: int64

In [100]:
mask = ((df['Address'] == 'MOCHI GATE LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5766)

mask = ((df['Address'] == 'MOCHI GATE LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3217)

In [101]:
mask = ((df['Address'] == 'ISLAM PURA LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5643)

mask = ((df['Address'] == 'ISLAM PURA LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.2986)

In [102]:
df.isna().sum()

Date                 0
Age                  0
Sex                  0
1st complaint        0
2nd Complaint        0
3rd Complaint        0
Address              0
Latitude         10528
Longitude        10528
dtype: int64

In [103]:
mask = ((df['Address'] == 'MOHNI RD LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.584838)

mask = ((df['Address'] == 'MOHNI RD LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.302591)

In [104]:
df.isna().sum()

Date                 0
Age                  0
Sex                  0
1st complaint        0
2nd Complaint        0
3rd Complaint        0
Address              0
Latitude         10383
Longitude        10383
dtype: int64

In [105]:
mask = ((df['Address'] == 'HALL RD LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5661)

mask = ((df['Address'] == 'HALL RD LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3185)

In [106]:
df['Address'] = df['Address'].replace(['KASUR', 'KASUR '], 'Kasur')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\1949125697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['KASUR', 'KASUR '], 'Kasur')


In [107]:
mask = ((df['Address'] == 'Kasur'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.60349)

mask = ((df['Address'] == 'Kasur'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.29838)

In [108]:
df['Address'] = df['Address'].replace(['MURIDKEY ', 'MUREEDKAY'], 'Muridke')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\40897358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['MURIDKEY ', 'MUREEDKAY'], 'Muridke')


In [109]:
mask = ((df['Address'] == 'Muridke'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.809484)

mask = ((df['Address'] == 'Muridke'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.253415)

In [110]:
df['Address'].value_counts()

Gawalmandi           38095
Shahdara              5429
Nisbat Road           4686
sheikhupura           3094
Badami Bagh           2301
                     ...  
IQBAL TOWN              52
JIA MUSA SHADRA         52
BHATI GATE LHR          51
DROUGHA WALA LHR        51
DAROOGANWALA            51
Name: Address, Length: 171, dtype: int64

In [111]:
mask = ((df['Address'] == 'GHARI SHAHU LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.55985)

mask = ((df['Address'] == 'GHARI SHAHU LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3369)

In [112]:
mask = ((df['Address'] == 'NEW CHAUBURJI PARK LAHORE'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.55283)

mask = ((df['Address'] == 'NEW CHAUBURJI PARK LAHORE'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.295688)

In [113]:
df['Address'] = df['Address'].replace(['BEGUM KOT SHAHDARA LHR', 'BAGHAM KOT SHADRA '], 'Begum Kot Shahdara Town')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\2728641587.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['BEGUM KOT SHAHDARA LHR', 'BAGHAM KOT SHADRA '], 'Begum Kot Shahdara Town')


In [114]:
mask = ((df['Address'] == 'Begum Kot Shahdara Town'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.58223)

mask = ((df['Address'] == 'Begum Kot Shahdara Town'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.35304)

In [115]:
df['Address'].value_counts()

Gawalmandi           38095
Shahdara              5429
Nisbat Road           4686
sheikhupura           3094
Badami Bagh           2301
                     ...  
IQBAL TOWN              52
JIA MUSA SHADRA         52
BHATI GATE LHR          51
DROUGHA WALA LHR        51
DAROOGANWALA            51
Name: Address, Length: 170, dtype: int64

In [116]:
df.isna().sum()

Date                0
Age                 0
Sex                 0
1st complaint       0
2nd Complaint       0
3rd Complaint       0
Address             0
Latitude         9285
Longitude        9285
dtype: int64

In [117]:
mask = ((df['Address'] == 'RANA TOWN '))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.685744)

mask = ((df['Address'] == 'RANA TOWN '))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.269284)

In [118]:
mask = ((df['Address'] == 'MULTAN RD LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.460494737)

mask = ((df['Address'] == 'MULTAN RD LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.22170044)

In [119]:
mask = ((df['Address'] == 'NARANG MANDI '))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.901405)

mask = ((df['Address'] == 'NARANG MANDI '))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.517032)

In [120]:
mask = ((df['Address'] == 'CHAH MEERA LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.587902)

mask = ((df['Address'] == 'CHAH MEERA LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.3426)

In [121]:
mask = ((df['Address'] == 'FAIZ BAGH LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.582586)

mask = ((df['Address'] == 'FAIZ BAGH LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.338977)

In [122]:
mask = ((df['Address'] == 'QILA GUJJAR SING LHR'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.567579)

mask = ((df['Address'] == 'QILA GUJJAR SING LHR'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.330153)

In [123]:
df.isna().sum()

Date                0
Age                 0
Sex                 0
1st complaint       0
2nd Complaint       0
3rd Complaint       0
Address             0
Latitude         8613
Longitude        8613
dtype: int64

In [124]:
mask = ((df['Address'] == 'SARGODHA '))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(32.073979)

mask = ((df['Address'] == 'SARGODHA '))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(72.68607)

In [125]:
mask = ((df['Address'] == 'NANKANA SAB'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.449151)

mask = ((df['Address'] == 'NANKANA SAB'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(73.712479)

In [126]:
pd.reset_option('display.max_rows', None)

In [127]:
df.isna().sum()

Date                0
Age                 0
Sex                 0
1st complaint       0
2nd Complaint       0
3rd Complaint       0
Address             0
Latitude         8403
Longitude        8403
dtype: int64

In [128]:
df

,Date,Age,Sex,1st complaint,2nd Complaint,3rd Complaint,Address,Latitude,Longitude
0,2011-05-01,18,2,fever,bodyaches,Headache,KOT ABDUL MALIK,31.62042,74.234381
1,2011-05-01,20,2,pain,bleeding,RTA,Gulshan-e-Ravi,31.55217,74.275290
2,2011-05-01,40,2,Dyspnea,chest pain,RTI,Gawalmandi,31.57187,74.318260
3,2011-05-01,24,2,headache,allergy,Vomiting,KOT ABDUL MALIK,31.62042,74.234381
4,2011-05-01,26,1,bleeding,pain,RTA,URDU BAZAR,NaN,NaN
...,...,...,...,...,...,...,...,...,...
83453,2011-08-31,35,1,Vomiting,fever,nausea,Anarkali Bazaar Lahore,31.56980,74.312000
83454,2011-08-31,35,1,Arthritis,Anxiety,arthralgia,Gawalmandi,31.57187,74.318260
83455,2011-08-31,18,2,Abdomen pain,Nausea,nausea,FAROOZWALA,NaN,NaN
83456,2011-08-31,46,1,Palpitation,anxiety,arthralgia,Lahori Gate,31.57741,74.313430


In [129]:
import re
df_iqbal = df[df['Address'].str.contains('Iqbal',flags=re.IGNORECASE)]
df_iqbal

,Date,Age,Sex,1st complaint,2nd Complaint,3rd Complaint,Address,Latitude,Longitude
1160,2011-05-02,18,2,Headache,chills,backache,IQBAL TOWN,NaN,NaN
1332,2011-05-02,20,2,Hypertension,vomiting,bodyaches,IQBAL TOWN LHR,NaN,NaN
1414,2011-05-02,52,2,severe pain in body,epigastric pain,kidney pain,IQBAL TOWN LHR,NaN,NaN
1873,2011-05-03,16,2,missed abortion,pain,vomiting,IQBAL TOWN LHR,NaN,NaN
1876,2011-05-03,20,2,fever,chills,headache,IQBAL TOWN LHR,NaN,NaN
...,...,...,...,...,...,...,...,...,...
82945,2011-08-31,50,1,dog bite,Anxiety,myalgia,IQBAL TOWN LHR,NaN,NaN
83103,2011-08-31,22,2,fever,headache,swelling,IQBAL TOWN LHR,NaN,NaN
83161,2011-08-31,20,1,Fever,Chills,Body pains,IQBAL TOWN LHR,NaN,NaN
83172,2011-08-31,30,2,Vomiting,pain abdomen,irritation,IQBAL TOWN LHR,NaN,NaN


In [130]:
df['Address'] = df['Address'].replace(['IQBAL TOWN', 'IQBAL TOWN'], 'Allama Iqbal Town')

C:\Users\drfak\AppData\Local\Temp\ipykernel_9316\959273631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address'] = df['Address'].replace(['IQBAL TOWN', 'IQBAL TOWN'], 'Allama Iqbal Town')


In [131]:
mask = ((df['Address'] == 'Allama Iqbal Town'))
df.loc[mask, 'Latitude'] = df.loc[mask, 'Latitude'].fillna(31.5124)

mask = ((df['Address'] == 'Allama Iqbal Town'))
df.loc[mask, 'Longitude'] = df.loc[mask, 'Longitude'].fillna(74.2845)

In [132]:
df['Address'].value_counts()

Gawalmandi           38095
Shahdara              5429
Nisbat Road           4686
sheikhupura           3094
Badami Bagh           2301
                     ...  
IQBAL TOWN              52
JIA MUSA SHADRA         52
BHATI GATE LHR          51
DROUGHA WALA LHR        51
DAROOGANWALA            51
Name: Address, Length: 170, dtype: int64

In [133]:
df.isna().sum()

Date                0
Age                 0
Sex                 0
1st complaint       0
2nd Complaint       0
3rd Complaint       0
Address             0
Latitude         8403
Longitude        8403
dtype: int64

In [134]:
df = df.dropna(axis=0)
df

,Date,Age,Sex,1st complaint,2nd Complaint,3rd Complaint,Address,Latitude,Longitude
0,2011-05-01,18,2,fever,bodyaches,Headache,KOT ABDUL MALIK,31.620420,74.234381
1,2011-05-01,20,2,pain,bleeding,RTA,Gulshan-e-Ravi,31.552170,74.275290
2,2011-05-01,40,2,Dyspnea,chest pain,RTI,Gawalmandi,31.571870,74.318260
3,2011-05-01,24,2,headache,allergy,Vomiting,KOT ABDUL MALIK,31.620420,74.234381
6,2011-05-01,20,2,pain,bleeding,RTA,Gawalmandi,31.571870,74.318260
...,...,...,...,...,...,...,...,...,...
83452,2011-08-31,31,2,Fever,Chills,Body pains,Gujranwala,32.166351,74.195900
83453,2011-08-31,35,1,Vomiting,fever,nausea,Anarkali Bazaar Lahore,31.569800,74.312000
83454,2011-08-31,35,1,Arthritis,Anxiety,arthralgia,Gawalmandi,31.571870,74.318260
83456,2011-08-31,46,1,Palpitation,anxiety,arthralgia,Lahori Gate,31.577410,74.313430


In [136]:
df.isna().sum()

Date             0
Age              0
Sex              0
1st complaint    0
2nd Complaint    0
3rd Complaint    0
Address          0
Latitude         0
Longitude        0
dtype: int64

In [137]:
df.to_csv('ABCD.csv', index = False, header = True)